### **Import VGG19 model**

In [1]:
import tensorflow as tf
from tensorflow.keras.applications.vgg19 import VGG19, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
import numpy as np
from PIL import Image

In [3]:

base_model = None
content_model = None
style_models = None
style_cache = {}


def initialize_models(content_layer='block5_conv2',
                      style_layers=None):
    global base_model, content_model, style_models

    if style_layers is None:
        style_layers = ['block1_conv1', 'block3_conv1', 'block5_conv1']

    print("Loading VGG19 model...")
    base_model = VGG19(include_top=False, weights='imagenet')
    base_model.trainable = False


    content_model = Model(
        inputs=base_model.input,
        outputs=base_model.get_layer(content_layer).output
    )


    style_models = [
        Model(
            inputs=base_model.input,
            outputs=base_model.get_layer(layer).output
        )
        for layer in style_layers
    ]

    print("Models loaded successfully")


def load_and_process_image(image_path_or_array, max_dim=512):


    if isinstance(image_path_or_array, str):
        img = Image.open(image_path_or_array)
    elif isinstance(image_path_or_array, Image.Image):
        img = image_path_or_array
    elif isinstance(image_path_or_array, np.ndarray):
        img = Image.fromarray(image_path_or_array.astype('uint8'))
    else:
        raise ValueError("Unsupported image input type")


    if img.mode == 'RGBA':
        img = img.convert('RGB')


    width, height = img.size
    scale = max_dim / max(width, height)
    if scale < 1:
        new_width = int(width * scale)
        new_height = int(height * scale)
        img = img.resize((new_width, new_height), Image.Resampling.LANCZOS)


    img_array = img_to_array(img)
    img_array = preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    return img_array


def deprocess_image(processed_img):

    if len(processed_img.shape) == 4:
        processed_img = np.squeeze(processed_img, axis=0)


    x = processed_img.copy()
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]


    x = np.clip(x, 0, 255).astype('uint8')

    return Image.fromarray(x)


def compute_content_cost(content_features, generated_features):

    return tf.reduce_mean(tf.square(content_features - generated_features))


def compute_gram_matrix(features):

    channels = int(features.shape[-1])
    a = tf.reshape(features, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)


def compute_style_cost(style_features_list, generated_features_list):

    style_weight = 1.0 / len(style_models)
    total_style_cost = 0.0

    for style_features, generated_features in zip(
        style_features_list, generated_features_list
    ):
        gram_style = compute_gram_matrix(style_features)
        gram_generated = compute_gram_matrix(generated_features)
        layer_cost = tf.reduce_mean(tf.square(gram_style - gram_generated))
        total_style_cost += style_weight * layer_cost

    return total_style_cost


def get_style_features(style_image, cache_key=None):

    global style_cache

    if cache_key and cache_key in style_cache:
        print(f"Using cached style features for '{cache_key}'")
        return style_cache[cache_key]

    features = [model(style_image) for model in style_models]

    if cache_key:
        style_cache[cache_key] = features
        print(f"Cached style features as '{cache_key}'")

    return features


def transfer_style(content_image, style_image,
                   iterations=300, alpha=1.0, beta=1e4,
                   learning_rate=5.0, style_cache_key=None,
                   progress_callback=None):

    print(f"Starting style transfer ({iterations} iterations)...")
    print(f"Alpha (content weight): {alpha}, Beta (style weight): {beta}")


    content_features = content_model(content_image)
    style_features = get_style_features(style_image, style_cache_key)


    generated = tf.Variable(content_image, dtype=tf.float32)


    optimizer = tf.optimizers.Adam(learning_rate=learning_rate)


    best_cost = float('inf')
    best_image = None


    for i in range(iterations):
        with tf.GradientTape() as tape:

            gen_content_features = content_model(generated)
            gen_style_features = [model(generated) for model in style_models]


            content_cost = compute_content_cost(
                content_features, gen_content_features
            )
            style_cost = compute_style_cost(
                style_features, gen_style_features
            )

            total_cost = alpha * content_cost + beta * style_cost


        gradients = tape.gradient(total_cost, generated)
        optimizer.apply_gradients([(gradients, generated)])

        if total_cost < best_cost:
            best_cost = total_cost
            best_image = generated.numpy()


        if progress_callback and (i % 10 == 0 or i == iterations - 1):
            progress_callback(i + 1, float(total_cost), generated.numpy())


        if i % 50 == 0:
            print(f"  Iteration {i}/{iterations} - Total: {total_cost:.2f}, Content: {content_cost:.2f}, Style: {style_cost:.2f}")

    print(f"Style transfer complete! Final cost: {best_cost:.2f}")

    return best_image, float(best_cost)


def apply_style(content_input, style_input,
                style_weight=0.5, iterations=300,
                max_dim=512, alpha=1.0, beta=None,
                learning_rate=5.0, style_cache_key=None,
                progress_callback=None):


    if base_model is None:
        initialize_models()


    content_image = load_and_process_image(content_input, max_dim)
    style_image = load_and_process_image(style_input, max_dim)


    if beta is None:
        beta = style_weight * 1e4


    result_array, final_cost = transfer_style(
        content_image=content_image,
        style_image=style_image,
        iterations=iterations,
        alpha=alpha,
        beta=beta,
        learning_rate=learning_rate,
        style_cache_key=style_cache_key,
        progress_callback=progress_callback
    )


    result_image = deprocess_image(result_array)

    return result_image



def quick_style_transfer(content_path, style_path, style_weight=0.5, iterations=300):

    result = apply_style(
        content_input=content_path,
        style_input=style_path,
        style_weight=style_weight,
        iterations=iterations
    )
    return result


def compare_style_weights(content_path, style_path, weights=[0.3, 0.5, 0.7], iterations=200):

    results = []
    for weight in weights:
        print(f"\n{'='*50}")
        print(f"Testing style_weight = {weight}")
        print('='*50)
        result = apply_style(
            content_input=content_path,
            style_input=style_path,
            style_weight=weight,
            iterations=iterations
        )
        results.append((weight, result))
    return results



if __name__ == "__main__":

    initialize_models()


    result = apply_style(
        content_input='japanese_garden.jpg',
        style_input='picasso_selfportrait.jpg',
        style_weight=0.5,
        iterations=300
    )

    result.save('result.jpg')
    print("Saved result to result.jpg")

Loading VGG19 model...
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
✓ Models loaded successfully!
Starting style transfer (300 iterations)...
Alpha (content weight): 1.0, Beta (style weight): 5000.0
  Iteration 0/300 - Total: 798199119872.00, Content: 0.00, Style: 159639824.00
  Iteration 50/300 - Total: 10835549184.00, Content: 3863.83, Style: 2167109.00
  Iteration 100/300 - Total: 5340538880.00, Content: 4019.80, Style: 1068107.00
  Iteration 150/300 - Total: 3937910784.00, Content: 4088.83, Style: 787581.31
  Iteration 200/300 - Total: 3194964480.00, Content: 4138.82, Style: 638992.06
  Iteration 250/300 - Total: 2712075008.00, Content: 4171.62, Style: 542414.19
✓ Style transfer complete! Final cost: 2371082240.00
Saved result to result.jpg
